In [10]:
from glotaran_tools.specification_parser import parse_yml
from glotaran_core.model import Dataset
from glotaran_models.kinetic import KineticSeperableModel
import numpy as np
from lmfit import Parameters
# Imports for plotting

import matplotlib.pyplot as plt
%matplotlib notebook

fitspec = '''
type: kinetic

parameter: {}

compartments: [s1, s2, s3]

initial_concentrations:
    - label: i1
      parameter: [4,5,6]

megacomplexes:
    - label: mc1
      k_matrices: [k1]

k_matrices:
  - label: "k1"
    matrix: {{
      '("s1","s1")': 1,
      '("s2","s2")': 2,
      '("s3","s3")': 3,
}}

irf: []

datasets: 
  - label: dataset1
    type: spectral
    initial_oncentration: i1
    megacomplexes: [mc1]
    path: 'data/dataset1.ascii'

'''
   

In [16]:
test_parameter = [101e-4, 202e-5, 505e-6]
test_amps = [1,2,3]
times = np.asarray(np.arange(0, 1500, 1.5))
simparams = Parameters()
simparams.add("p1", 101e-3)
simparams.add("p2",202e-4)
simparams.add("p3",505e-5)

In [17]:
        
model = parse_yml(fitspec.format(test_parameter))

In [18]:
fitmodel = KineticSeperableModel(model)

In [19]:
print(fitmodel.e_matrix(**{'dataset':'dataset1'}))
data = fitmodel.eval(simparams, *times, **{'dataset':'dataset1'})

[[ 1.  1.  1.]]


In [20]:
c = fitmodel.c_matrix(fitmodel.get_initial_fitting_parameter().valuesdict(), *times)
print(c.shape)

(1000, 3)


In [21]:
result = fitmodel.fit(fitmodel.get_initial_fitting_parameter(), 
                      *times, 
                      **{"data": data}
                     )
print(result.params)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.9618e+00                                    3.37e+02    
       1              3         1.3170e+00      6.45e-01       2.58e-03       1.77e+02    
       2              4         8.5563e-01      4.61e-01       5.16e-03       9.93e+01    
       3              5         4.2961e-01      4.26e-01       1.03e-02       2.34e+02    
       4              6         1.2556e-01      3.04e-01       1.19e-02       9.07e+01    
       5              7         7.4509e-02      5.11e-02       7.54e-03       3.94e+00    
       6              8         6.2819e-02      1.17e-02       6.00e-03       2.56e+01    
       7             10         4.4607e-02      1.82e-02       1.76e-03       6.39e+00    
       8             11         3.9570e-02      5.04e-03       3.52e-03       1.88e+01    
       9             12         2.7290e-02      1.23e-02       3.52e-03       6.90e+00    